## Generating Atoms Models

Importing SymbolInfo Class.

In [ ]:
from wizard.atoms import SymbolInfo

Creating bulk atoms.

In [ ]:
atoms = SymbolInfo('W', 'bcc', 3).create_bulk_atoms()

Importing Morph Class.

In [ ]:
from wizard.atoms import Morph

Creating HEA model.

In [ ]:
Morph(atoms).prop_element_set(['Mo','Nb','Ta','V','W'])

Creating mono-vacancy model.

In [ ]:
Morph(atoms).create_vacancy()

Creating di-vacancies model.

In [ ]:
Morph(atoms).create_divacancies()

Creating self-interstitial-atom model.

In [ ]:
Morph(atoms).create_self_interstitial_atom([1, 1, 1])

Creating interstitial-atom model.

In [ ]:
from ase import Atom
atom = Atom('W', position=(0, 0, 0))
atoms.append(atom)

Creating Frenkel defects model.

In [ ]:
Morph(atoms).create_fks(10)

## Molecular dynamics

Run molecular dynamics (MD) simulations using [GPUMD](https://github.com/brucefan1983/GPUMD).

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

Running molecular dynamics simulations.

In [ ]:
run_in = ['potential ../nep.txt', 
          'velocity 300', 
          'time_step 1', 
          'ensemble npt_scr 300 300 200 0 500 2000',
          'dump_thermo 1000', 
          'dump_restart 30000', 
          'dump_exyz 10000',
          'run 30000']
Morph(atoms).gpumd('relax', run_in)

Deforming the simulation box.

In [ ]:
run_in = ['potential ../nep.txt', 
          'velocity 300', 
          'time_step 1',
          'ensemble npt_scr 300 300 100 0 0 0 100 100 100 1000',
          'run 30000', 
          'ensemble npt_scr 300 300 100 0 0 0 100 100 100 1000',
          'deform 0.00001 0 0 1', 
          'dump_thermo 1000', 
          'dump_exyz 1000', 
          'dump_restart 10000',
          'run 1000000']
Morph(atoms).gpumd('deform', run_in)

Simulating the process of a crystallization.

In [ ]:
run_in = ['potential ../nep.txt', 
          'velocity 2000', 
          'time_step 1', 
          'ensemble npt_scr 2000 2000 200 0 500 2000', 
          'dump_thermo 1000', 
          'dump_xyz 10000', 
          'dump_restart 10000', 
          'run 100000',
          'ensemble npt_scr 2000 5000 200 0 500 2000',
          'dump_thermo 1000', 
          'dump_xyz 100000', 
          'dump_restart 10000', 
          'run 10000000',
          'ensemble npt_scr 4500 4500 200 0 500 2000',
          'dump_thermo 1000', 
          'dump_xyz 10000', 
          'dump_restart 10000', 
          'run 100000',
          'ensemble npt_scr 4500 1500 200 0 500 2000',
          'dump_thermo 1000', 
          'dump_xyz 100000', 
          'dump_restart 10000', 
          'run 10000000']
Morph(atoms).gpumd('crystallization', run_in)

Calculating the Melting point using two-phase coexistence method.

In [ ]:
from wizard.io import read_xyz

group = []
for atom in atoms:
    if atom.position[2] < atoms.cell[2, 2] / 2:
        group.append(0)
    else:
        group.append(1)
atoms.info['group'] = group

run_in_1 = ['potential ../../nep.txt', 
            'velocity 3000', 
            'time_step 1', 
            'ensemble npt_ber 3000 3000 200 0 500 2000', 
            'dump_exyz 10000', 
            'dump_thermo 1000',
            'run 30000',
            'ensemble heat_lan 3500 200 500 0 1',
            'dump_exyz 10000',
            'dump_thermo 1000',
            'dump_restart 10000',
            'run 1000000']

Morph(atoms).gpumd('melting_point/relax', run_in_1)

for Tm in range(3400, 3701, 100):
    atoms = read_xyz('melting_point/relax/dump.xyz')[-1]
    run_in = ['potential ../../nep.txt', 
             f'velocity {Tm}', 
              'time_step 1', 
             f'ensemble npt_ber {Tm} {Tm} 200 0 500 2000', 
              'dump_exyz 10000', 
              'dump_thermo 1000',
              'run 30000']
    Morph(atoms).gpumd(f'melting_point/{Tm}', run_in)

Simulating the radiation damage.

In [ ]:
from wizard.io import read_xyz
import numpy as np

group = []
thickness = 3.185 * 3
for atom in atoms:
    if atom.position[0] < thickness or atom.position[1] < thickness or atom.position[2] < thickness:
        group.append(0)
    elif atom.position[0] >= atoms.cell[0, 0] - thickness or atom.position[1] >= atoms.cell[1, 1] - thickness or atom.position[2] >= atoms.cell[2, 2] - thickness:
        group.append(1)
    else:
        group.append(2)
atoms.info['group'] = group

run_in_1 = ['potential ../../nep.txt',
            'velocity 300', 
            'time_step 1', 
            'ensemble npt_scr 300 300 200 0 500 2000', 
            'dump_thermo 1000', 
            'dump_restart 30000', 
            'run 30000']

run_in_2 = ['potential ../../nep.txt', 
            'velocity 300', 
            'time_step 0', 
            'ensemble nve',
            'dump_exyz 1', 
            'run 1',
            'time_step 1 0.015', 
            'ensemble heat_nhc 300 200 0 0 1',
            'electron_stop ../../electron_stopping_fit.txt',
            'compute 0 200 10 temperature', 
            'dump_restart 10000', 
            'dump_exyz 2000 1 1',
            'run 70000']

pka_energy = 10 #eV
direction = np.array([1, 3, 5]) 
index = 100

Morph(atoms).gpumd('radiation/relax', run_in_1)
atoms = read_xyz('radiation/relax/restart.xyz')[-1]
Morph(atoms).set_pka(pka_energy, direction, index)
Morph(atoms).gpumd('radiation/cascade', run_in_2)

Simulating the overlapping cascades.

In [ ]:
from wizard.io import read_restart
import numpy as np
import random

run_in_1 = ['potential ../../nep.txt',
            'velocity 300', 
            'time_step 1', 
            'ensemble npt_scr 300 300 200 0 500 2000', 
            'dump_thermo 1000', 
            'dump_restart 10000', 
            'run 30000']

run_in_2 = ['potential ../../nep.txt',
            'velocity 300', 
            'time_step 1', 
            'ensemble npt_scr 300 300 200 0 500 2000', 
            'dump_thermo 1000', 
            'dump_restart 10000', 
            'run 10000']

run_in_3 = ['potential ../../nep.txt', 
            'velocity 300', 
            'time_step 0', 
            'ensemble nve',
            'dump_exyz 1', 
            'run 1',
            'time_step 1 0.015', 
            'ensemble heat_nhc 300 200 0 0 1',
            'electron_stop ../../electron_stopping_fit.txt',
            'compute 0 200 10 temperature', 
            'dump_restart 10000', 
            'dump_exyz 2000 1 1',
            'run 40000']

pka_energy = 10 #eV
cascade_times = 2000
directions = [np.array([np.sin(np.random.uniform(0, np.pi)) * np.cos(np.random.uniform(0, 2 * np.pi)),
                        np.sin(np.random.uniform(0, np.pi)) * np.sin(np.random.uniform(0, 2 * np.pi)),
                        np.cos(np.random.uniform(0, np.pi))]) for _ in range(cascade_times)]

indexs = [random.randint(0, len(atoms) - 1) for _ in range(cascade_times)]

## First time
direction = directions[0]
index = indexs[0]

center = atoms.cell.diagonal() / 2
diff = center - atoms[index].position
for atom in atoms:
    atom.position += diff

for atom in atoms:
    atom.position %= atoms.cell.diagonal()

group = []
thickness = 3.185 * 3
for atom in atoms:
    if atom.position[0] < thickness or atom.position[1] < thickness or atom.position[2] < thickness:
        group.append(0)
    elif atom.position[0] >= atoms.cell[0, 0] - thickness or atom.position[1] >= atoms.cell[1, 1] - thickness or atom.position[2] >= atoms.cell[2, 2] - thickness:
        group.append(1)
    else:
        group.append(2)
atoms.info['group'] = group

Morph(atoms).gpumd('radiation0/relax', run_in_1)
atoms = read_restart('radiation0/relax/restart.xyz')
Morph(atoms).set_pka(pka_energy, direction, index)
Morph(atoms).gpumd('radiation0/cascade', run_in_3)

## Loops
for i in range(1, cascade_times):
    direction = directions[i]
    index = indexs[i]
    atoms = read_restart(f'radiation{i-1}/cascade/restart.xyz')

    center = atoms.cell.diagonal() / 2
    diff = center - atoms[index].position
    for atom in atoms:
        atom.position += diff

    for atom in atoms:
        atom.position %= atoms.cell.diagonal()

    group = []
    thickness = 3.185 * 3
    for atom in atoms:
        if atom.position[0] < thickness or atom.position[1] < thickness or atom.position[2] < thickness:
            group.append(0)
        elif atom.position[0] >= atoms.cell[0, 0] - thickness or atom.position[1] >= atoms.cell[1, 1] - thickness or atom.position[2] >= atoms.cell[2, 2] - thickness:
            group.append(1)
        else:
            group.append(2)
    atoms.info['group'] = group

    Morph(atoms).gpumd(f'radiation{i}/relax', run_in_2)
    atoms = read_restart(f'radiation{i}/relax/restart.xyz')
    Morph(atoms).set_pka(pka_energy, direction, index)
    Morph(atoms).gpumd(f'radiation{i}/cascade', run_in_3)

Calculate the threshold displacement energy surface

In [ ]:
import numpy as np

group = []
thickness = 3.185
for atom in atoms:
    if atom.position[0] < thickness or atom.position[1] < thickness or atom.position[2] < thickness:
        group.append(0)
    elif atom.position[0] >= atoms.cell[0, 0] - thickness or atom.position[1] >= atoms.cell[1, 1] - thickness or atom.position[2] >= atoms.cell[2, 2] - thickness:
        group.append(1)
    else:
        group.append(2)
        
atoms.info['group'] = group

theta_range = np.arange(0, 46, 5) 
phi_range = np.arange(0, 46, 5) 

directions = []

for theta in theta_range:
    for phi in phi_range:
        theta_rad = np.deg2rad(theta)
        phi_rad = np.deg2rad(phi)
        x = np.sin(theta_rad) * np.cos(phi_rad)
        y = np.sin(theta_rad) * np.sin(phi_rad)
        z = np.cos(theta_rad)
        directions.append(np.array([x, y, z]))

run_in = ['potential ../../nep.txt', 
          'velocity 1', 
          'time_step 0', 
          'ensemble nve',
          'dump_exyz 1', 
          'run 1',
          'time_step 1 0.015', 
          'ensemble heat_nhc 10 200 0 0 1',
          'electron_stop ../../electron_stopping_fit.txt',
          'dump_exyz 500 1 1',
          'run 5000']

atoms.info['velocities'] = np.zeros((len(atoms), 3))

for direction in directions:
    for pka_energy in range(10, 101, 10):
        index = 8420
        Morph(atoms).set_pka(pka_energy, direction, index)
        Morph(atoms).gpumd(f'{direction}/{pka_energy}', run_in)

## Calculating Material Properties

Using the calculator from [PyNEP](https://github.com/bigd4/PyNEP).

In [ ]:
from wizard.calculator import MaterialCalculator
from pynep.calculate import NEP

calc = NEP('train/nep.txt')
material_calculator = MaterialCalculator(atoms, calc, 'Nb', 'bcc')

Calculate the lattice constants of the material, write them along with the atom energy to a file, and return the atom energy and cell lengths.

In [ ]:
material_calculator.lattice_constant()

Calculate the elastic constants of the material, write them to a file, and return the bulk modulus.

In [ ]:
material_calculator.elastic_constant()

Generate the equation of state (EOS) curve for the material, save the curve as a PNG image, and write the volume and energy data to a file.

In [ ]:
material_calculator.eos_curve()

Calculate and plot the phonon dispersion band structure for the material.

In [ ]:
material_calculator.phonon_dispersion()

Calculate the formation energy of a vacancy in the material, write it to a file, and return the formation energy.

In [ ]:
material_calculator.formation_energy_vacancy(relax_params={'fmax':0.001})

Calculate the migration energy of a vacancy in the material by simulating the transition from the initial to the final state.

In [ ]:
material_calculator.migration_energy_vacancy()

Calculate the formation energy of divacancies in the material, write it to a file, and return the formation energy.

In [ ]:
nth = 1
material_calculator.formation_energy_divacancies(nth, relax_params={'fmax':0.001})

Calculate the formation energy of a self-interstitial atom (SIA) in the material, write it to a file, and return the formation energy.

In [ ]:
vector = (1,1,1)
material_calculator.formation_energy_sia(vector)

Calculate the formation energy of an interstitial atom in the material, write it to a file, and return the formation energy.

In [ ]:
material_calculator.formation_energy_interstitial_atom('W',[0,0,1/2],'octahedral')

Calculate the formation energy of a surface in the material, write it to a file, and return the formation energy in meV.

In [ ]:
miller = (0, 0, 1)
material_calculator.formation_energy_surface(miller)

Calculate the energy of a stacking fault in the material by simulating the shift of atomic layers, write the maximum energy to a file, plot the energy as a function of shift distance, and return the energy values.

In [ ]:
material_calculator.stacking_fault(a = (1,1,-1), b = (1,-1,0), miller = [1,1,2], distance = 3.185/2)

Simulate the movement of a screw dislocation dipole in a pure BCC metal, calculate the energy at each step, save the energy profile as a PNG image, and return the energy values.

In [ ]:
material_calculator.pure_bcc_metal_screw_dipole_move()

Simulate the movement of a single screw dislocation in a pure BCC metal, calculate the energy at each step, save the energy profile as a PNG image, and return the energy values.

In [ ]:
material_calculator.pure_bcc_metal_screw_one_move()

## Tools

Reading atomic configuraion and caculating the potential energy and forces using NEP. The results are then plotted and the Root Mean Square Error (RMSE) is computed.

In [ ]:
from wizard.io import read_xyz, plot_e, plot_f
from pynep.calculate import NEP
import numpy as np

frames = read_xyz('../HEA-Jesper/train/train.xyz') 
print(len(frames))
calc = NEP('train/nep.txt')
ed, en, fd, fn = [], [], [], []
for atoms in frames:
    atoms.calc = calc
    en.append(atoms.get_potential_energy() / len(atoms))
    ed.append(atoms.info['energy'] / len(atoms))
    fn.append(atoms.get_forces())
    fd.append(atoms.info['forces'])
ed = np.array(ed)
en = np.array(en)
fd = np.concatenate(fd)
fn = np.concatenate(fn)
plot_e(ed, en)
plot_f(fd, fn)
e_rmse = np.sqrt(np.mean((ed-en)**2)) 
f_rmse = np.sqrt(np.mean((fd-fn)**2))
print(e_rmse)
print(f_rmse)

In [ ]:
from wizard.io import read_xyz, plot_e, plot_f
from pynep.calculate import NEP
import numpy as np

frames = read_xyz('../HEA-Jesper/train/train.xyz') 
print(len(frames))
calc = NEP('../HEA-Jesper/train/nep.txt')
ed, en, fd, fn = [], [], [], []
for atoms in frames:
    atoms.calc = calc
    en.append(atoms.get_potential_energy() / len(atoms))
    ed.append(atoms.info['energy'] / len(atoms))
    fn.append(atoms.get_forces())
    fd.append(atoms.info['forces'])
ed = np.array(ed)
en = np.array(en)
fd = np.concatenate(fd)
fn = np.concatenate(fn)
plot_e(ed, en)
plot_f(fd, fn)
e_rmse = np.sqrt(np.mean((ed-en)**2)) 
f_rmse = np.sqrt(np.mean((fd-fn)**2))
print(e_rmse)
print(f_rmse)

## Generating Train Set

Creating Atom Models with Different Crystal Structures.

In [1]:
from wizard.atoms import SymbolInfo, Morph
import numpy as np

train_set = []
SymbolInfos = [
   SymbolInfo('WMo', 'bcc', 3.32),
]

Generating and Saving Deformed Atom Models with Random Strain and Displacement.

In [ ]:
from wizard.generator import Generator

frames = []
for SymbolInfo in SymbolInfos:
    atoms = SymbolInfo.create_bulk_atoms((2, 2, 2))
    frames.append(atoms)

deform_scale = np.arange(0.95, 1.06, 0.05)
strain_ratio = 0.04
max_displacement = 0.4

init_1 = Generator(frames).deform(deform_scale)
init_2 = Generator(init_1).random_strain(strain_ratio)
init_3 = Generator(init_1).random_displacement(max_displacement)

init = init_1 + init_2 + init_3

Generating Atom Models with Various Types of Defects

In [ ]:
from ase import Atoms

dimmers = []
distances = [1 + i * 0.5 for i in range(10)]
for symbol_info in SymbolInfos:
    symbol1 = symbol_info.symbols[0]
    symbol2 = symbol_info.symbols[-1]
    for distance in distances:
        dimmer = Atoms([symbol1, symbol2], positions=[(0, 0, 0), (0, 0, distance)])
        dimmers.append(dimmer)

In [ ]:
mono_vacancy = []
for symbol_info in SymbolInfos:
    atoms = symbol_info.create_bulk_atoms((3, 4, 5))
    Morph(atoms).create_vacancy()
    mono_vacancy.append(atoms)

In [ ]:
nths = [1,2,3,4,5]
di_vacancies = []
for symbol_info in SymbolInfos:
    for nth in nths:
        atoms = symbol_info.create_bulk_atoms((3, 4, 5)) 
        Morph(atoms).create_divacancies(nth)
        di_vacancies.append(atoms)

In [ ]:
vacancies = []
for symbol_info in SymbolInfos:
    for n in range(3, 10, 3):
        atoms = symbol_info.create_bulk_atoms((3, 4, 5)) 
        Morph(atoms).create_vacancies(n)
        vacancies.append(atoms)

In [ ]:
sia = []
vectors = [(1,1,1),(1,0,0),(1,1,0)]
for symbol_info in SymbolInfos:
    for vector in vectors:
        atoms = symbol_info.create_bulk_atoms((3,4,5)) 
        Morph(atoms).create_self_interstitial_atom(vector)
        sia.append(atoms)

In [ ]:
vectors = [(1,1,1),(1,0,0),(1,1,0)]
nths = [1,2,3,4,5]
di_sias = []
for symbol_info in SymbolInfos:
    for vector1 in vectors:
        for vector2 in vectors:
            for nth in nths:
                atoms = symbol_info.create_bulk_atoms((3,4,5))
                Morph(atoms).create_di_self_interstitial_atoms(vector1=vector1, vector2=vector2, nth=nth)
                di_sias.append(atoms)

In [ ]:
fks = []
for symbol_info in SymbolInfos:
    atoms = symbol_info.create_bulk_atoms((3,4,5))
    Morph(atoms).create_fks(10)
    fks.append(atoms)

In [ ]:
from ase.build import surface

millers = [(1,1,0),(0,0,1),(1,1,1),(1,1,2)]
surf = []
for symbol_info in SymbolInfos:
    for miller in millers:
        atoms = symbol_info.create_bulk_atoms()
        slab = surface(atoms, miller, layers = 10, vacuum=10) *(2, 2, 1)
        surf.append(slab)